In [5]:
!pip install torch torchvision


#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv

# 安装torch_geometric
!pip install torch-geometric



In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive', force_remount=True)

# 读取CSV文件
df = pd.read_csv('/content/drive/My Drive/entity_neighbors_results.csv')

# 打印前几行数据帧
print(df.head())

# 构建图数据
def build_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        tweet_id = row['Tweet ID']
        entity = row['Entity']
        neighbor_label = row['Neighbor Label']
        is_fake_news = row['Is Fake News'] == 'rumor'

        G.add_node(tweet_id, label=is_fake_news)  # 添加推文节点
        G.add_node(entity, tweet_id=tweet_id)  # 添加实体节点
        G.add_edge(tweet_id, entity)  # 推文和实体之间的边

        if neighbor_label:
            G.add_node(neighbor_label)
            G.add_edge(entity, neighbor_label)  # 实体和邻居之间的边

    return G

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 构建训练集和测试集的图结构
train_graph = build_graph(train_df)
test_graph = build_graph(test_df)

# 加载GloVe词向量
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = torch.tensor([float(val) for val in values[1:]], dtype=torch.float32)
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line: {line}")
    return embeddings_index

glove_embeddings = load_glove_embeddings('/content/drive/My Drive/glove.42B.300d.txt')

# 准备数据
def prepare_data(G, glove_embeddings):
    features = []
    labels = []
    valid_nodes = set()

    # 获取节点索引映射
    node_index = {node: i for i, node in enumerate(G.nodes)}

    for node in G.nodes:
        if 'label' in G.nodes[node]:
            labels.append(int(G.nodes[node]['label']))
            if node in glove_embeddings:
                glove_feature = glove_embeddings[node]
            else:
                glove_feature = torch.randn(300)  # 如果找不到词向量，则使用随机向量
            features.append(glove_feature)
            valid_nodes.add(node)

    # 构建edge_index，过滤掉与没有标签节点相关的边
    edge_index = []
    for u, v in G.edges:
        if u in valid_nodes and v in valid_nodes:
            edge_index.append([node_index[u], node_index[v]])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.stack(features)
    y = torch.tensor(labels, dtype=torch.long)

    print(f"x.shape: {x.shape}, edge_index.shape: {edge_index.shape}, y.shape: {y.shape}")
    print(f"Number of nodes: {G.number_of_nodes()}, Number of edges: {G.number_of_edges()}")

    return Data(x=x, edge_index=edge_index, y=y)

# 获取节点特征
train_data = prepare_data(train_graph, glove_embeddings)
test_data = prepare_data(test_graph, glove_embeddings)

# 设置掩码
train_data.train_mask = torch.tensor([True] * train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.tensor([True] * test_data.num_nodes, dtype=torch.bool)

# 创建数据加载器
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

# 确保数据在同一设备上
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = train_data.to(device)
test_data = test_data.to(device)


Mounted at /content/drive
             Tweet ID  Entity                               Wikidata Result  \
0  498254340310966273   today            Entity found: L3283 with ID: L3283   
1  498254340310966273   today            Entity found: L3283 with ID: L3283   
2  498254929942028288     ten  Entity found: L348170-S1 with ID: L348170-S1   
3  498294367405477888     ten  Entity found: L348170-S1 with ID: L348170-S1   
4  498305825341845504  monday  Entity found: L345300-S1 with ID: L345300-S1   

                             Neighbor ID Neighbor Label Is Fake News  
0   http://www.wikidata.org/entity/Q1860        English        rumor  
1   http://www.wikidata.org/entity/Q1084           noun        rumor  
2  http://www.wikidata.org/entity/Q23806             10        rumor  
3  http://www.wikidata.org/entity/Q23806             10        rumor  
4    http://www.wikidata.org/entity/Q105         Monday        rumor  
x.shape: torch.Size([963, 300]), edge_index.shape: torch.Size([0]), y.sha

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
#调试1
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive', force_remount=True)

# 读取CSV文件
df = pd.read_csv('/content/drive/My Drive/entity_neighbors_results.csv')

# 打印前几行数据帧
print(df.head())

# 构建图
def build_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        tweet_id = row['Tweet ID']
        entity = row['Entity']
        neighbor_label = row['Neighbor Label']
        is_fake_news = row['Is Fake News'] == 'rumor'

        G.add_node(tweet_id, label=is_fake_news)  # 添加推文节点
        G.add_node(entity, tweet_id=tweet_id)  # 添加实体节点
        G.add_edge(tweet_id, entity)  # 推文和实体之间的边

        if neighbor_label:
            G.add_node(neighbor_label)
            G.add_edge(entity, neighbor_label)  # 实体和邻居之间的边

    print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 构建训练集和测试集的图结构
train_graph = build_graph(train_df)
test_graph = build_graph(test_df)

# 加载GloVe词向量
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = torch.tensor([float(val) for val in values[1:]], dtype=torch.float32)
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line: {line}")
    return embeddings_index

glove_embeddings = load_glove_embeddings('/content/drive/My Drive/glove.42B.300d.txt')

# 准备数据
def prepare_data(G, glove_embeddings):
    features = []
    labels = []
    valid_nodes = set()

    # 获取节点索引映射
    node_index = {node: i for i, node in enumerate(G.nodes)}

    for node in G.nodes:
        if 'label' in G.nodes[node]:
            labels.append(int(G.nodes[node]['label']))
            if node in glove_embeddings:
                glove_feature = glove_embeddings[node]
            else:
                glove_feature = torch.randn(300)  # 如果找不到词向量，则使用随机向量
            features.append(glove_feature)
            valid_nodes.add(node)

    print(f"Valid nodes: {valid_nodes}")

    # 构建edge_index，过滤掉与没有标签节点相关的边
    edge_index = []
    for u, v in G.edges:
        if u in valid_nodes and v in valid_nodes:
            edge_index.append([node_index[u], node_index[v]])

    if not edge_index:
        print("Warning: No edges found. Check the graph construction logic.")

    print(f"Edge index before conversion: {edge_index}")

    if edge_index:  # Ensure edge_index is not empty before converting to tensor
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)

    x = torch.stack(features)
    y = torch.tensor(labels, dtype=torch.long)

    print(f"x.shape: {x.shape}, edge_index.shape: {edge_index.shape}, y.shape: {y.shape}")
    print(f"Number of nodes: {G.number_of_nodes()}, Number of edges: {G.number_of_edges()}")

    return Data(x=x, edge_index=edge_index, y=y)

# 获取节点特征
train_data = prepare_data(train_graph, glove_embeddings)
test_data = prepare_data(test_graph, glove_embeddings)

# 设置掩码
train_data.train_mask = torch.tensor([True] * train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.tensor([True] * test_data.num_nodes, dtype=torch.bool)

# 创建数据加载器
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

# 确保数据在同一设备上
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = train_data.to(device)
test_data = test_data.to(device)


Mounted at /content/drive
             Tweet ID  Entity                               Wikidata Result  \
0  498254340310966273   today            Entity found: L3283 with ID: L3283   
1  498254340310966273   today            Entity found: L3283 with ID: L3283   
2  498254929942028288     ten  Entity found: L348170-S1 with ID: L348170-S1   
3  498294367405477888     ten  Entity found: L348170-S1 with ID: L348170-S1   
4  498305825341845504  monday  Entity found: L345300-S1 with ID: L345300-S1   

                             Neighbor ID Neighbor Label Is Fake News  
0   http://www.wikidata.org/entity/Q1860        English        rumor  
1   http://www.wikidata.org/entity/Q1084           noun        rumor  
2  http://www.wikidata.org/entity/Q23806             10        rumor  
3  http://www.wikidata.org/entity/Q23806             10        rumor  
4    http://www.wikidata.org/entity/Q105         Monday        rumor  
Graph built with 1586 nodes and 1799 edges
Graph built with 896 nodes and

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
#调试2
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive', force_remount=True)

# 读取CSV文件
df = pd.read_csv('/content/drive/My Drive/entity_neighbors_results.csv')

# 打印前几行数据帧
print(df.head())

# 构建图数据
def build_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        tweet_id = row['Tweet ID']
        entity = row['Entity']
        neighbor_label = row['Neighbor Label']
        is_fake_news = row['Is Fake News'] == 'rumor'

        G.add_node(tweet_id, label=is_fake_news)  # 添加推文节点
        G.add_node(entity, tweet_id=tweet_id)  # 添加实体节点
        G.add_edge(tweet_id, entity)  # 推文和实体之间的边

        if neighbor_label:
            G.add_node(neighbor_label)
            G.add_edge(entity, neighbor_label)  # 实体和邻居之间的边

    print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 构建训练集和测试集的图结构
train_graph = build_graph(train_df)
test_graph = build_graph(test_df)

# 加载GloVe词向量
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = torch.tensor([float(val) for val in values[1:]], dtype=torch.float32)
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line: {line}")
    return embeddings_index

glove_embeddings = load_glove_embeddings('/content/drive/My Drive/glove.42B.300d.txt')

# 准备数据
def prepare_data(G, glove_embeddings):
    features = []
    labels = []
    valid_nodes = set()

    # 获取节点索引映射
    node_index = {node: i for i, node in enumerate(G.nodes)}

    for node in G.nodes:
        if 'label' in G.nodes[node]:
            labels.append(int(G.nodes[node]['label']))
            if node in glove_embeddings:
                glove_feature = glove_embeddings[node]
            else:
                glove_feature = torch.randn(300)  # 如果找不到词向量，则使用随机向量
            features.append(glove_feature)
            valid_nodes.add(node)

    print(f"Valid nodes: {valid_nodes}")

    # 构建edge_index，过滤掉与没有标签节点相关的边
    edge_index = []
    for u, v in G.edges:
        if u in valid_nodes and v in valid_nodes:
            edge_index.append([node_index[u], node_index[v]])
        else:
            print(f"Filtered edge: ({u}, {v})")

    if not edge_index:
        print("Warning: No edges found. Check the graph construction logic.")

    print(f"Edge index before conversion: {edge_index}")

    if edge_index:  # Ensure edge_index is not empty before converting to tensor
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)

    x = torch.stack(features)
    y = torch.tensor(labels, dtype=torch.long)

    print(f"x.shape: {x.shape}, edge_index.shape: {edge_index.shape}, y.shape: {y.shape}")
    print(f"Number of nodes: {G.number_of_nodes()}, Number of edges: {G.number_of_edges()}")

    return Data(x=x, edge_index=edge_index, y=y)

# 获取节点特征
train_data = prepare_data(train_graph, glove_embeddings)
test_data = prepare_data(test_graph, glove_embeddings)

# 设置掩码
train_data.train_mask = torch.tensor([True] * train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.tensor([True] * test_data.num_nodes, dtype=torch.bool)

# 创建数据加载器
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

# 确保数据在同一设备上
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = train_data.to(device)
test_data = test_data.to(device)


Mounted at /content/drive
             Tweet ID  Entity                               Wikidata Result  \
0  498254340310966273   today            Entity found: L3283 with ID: L3283   
1  498254340310966273   today            Entity found: L3283 with ID: L3283   
2  498254929942028288     ten  Entity found: L348170-S1 with ID: L348170-S1   
3  498294367405477888     ten  Entity found: L348170-S1 with ID: L348170-S1   
4  498305825341845504  monday  Entity found: L345300-S1 with ID: L345300-S1   

                             Neighbor ID Neighbor Label Is Fake News  
0   http://www.wikidata.org/entity/Q1860        English        rumor  
1   http://www.wikidata.org/entity/Q1084           noun        rumor  
2  http://www.wikidata.org/entity/Q23806             10        rumor  
3  http://www.wikidata.org/entity/Q23806             10        rumor  
4    http://www.wikidata.org/entity/Q105         Monday        rumor  
Graph built with 1586 nodes and 1799 edges
Graph built with 896 nodes and

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [7]:
#调试3
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from google.colab import drive

# 挂载Google Drive
drive.mount('/content/drive', force_remount=True)

# 读取CSV文件
df = pd.read_csv('/content/drive/My Drive/entity_neighbors_results.csv')

# 打印前几行数据帧
print(df.head())

# 构建图数据
def build_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        tweet_id = row['Tweet ID']
        entity = row['Entity']
        neighbor_label = row['Neighbor Label']
        is_fake_news = row['Is Fake News'] == 'rumor'

        G.add_node(tweet_id, label=is_fake_news)  # 添加推文节点
        G.add_node(entity, tweet_id=tweet_id)  # 添加实体节点
        G.add_edge(tweet_id, entity)  # 推文和实体之间的边

        if neighbor_label:
            G.add_node(neighbor_label)
            G.add_edge(entity, neighbor_label)  # 实体和邻居之间的边

    print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 构建训练集和测试集的图结构
train_graph = build_graph(train_df)
test_graph = build_graph(test_df)

# 加载GloVe词向量
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = torch.tensor([float(val) for val in values[1:]], dtype=torch.float32)
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line: {line}")
    return embeddings_index

glove_embeddings = load_glove_embeddings('/content/drive/My Drive/glove.42B.300d.txt')

# 准备数据
def prepare_data(G, glove_embeddings):
    features = []
    labels = []
    valid_nodes = set()

    # 获取节点索引映射
    node_index = {node: i for i, node in enumerate(G.nodes)}

    # 确保每个节点的标签和特征正确
    for node in G.nodes:
        if 'label' in G.nodes[node]:
            labels.append(int(G.nodes[node]['label']))
            if node in glove_embeddings:
                glove_feature = glove_embeddings[node]
            else:
                glove_feature = torch.randn(300)  # 如果找不到词向量，则使用随机向量
            features.append(glove_feature)
            valid_nodes.add(node)
            print(f"Node {node}: label={G.nodes[node]['label']}, feature_shape={glove_feature.shape}")
        else:
            print(f"Node {node} has no label and will be skipped.")

    print(f"Valid nodes: {valid_nodes}")
    print(f"Node index mapping: {node_index}")

    # 构建edge_index，过滤掉与没有标签节点相关的边
    edge_index = []
    for u, v in G.edges:
        if u in valid_nodes and v in valid_nodes:
            edge_index.append([node_index[u], node_index[v]])
        else:
            print(f"Filtered edge: ({u}, {v})")

    if not edge_index:
        print("Warning: No edges found. Check the graph construction logic.")

    print(f"Edge index before conversion: {edge_index}")

    if edge_index:  # Ensure edge_index is not empty before converting to tensor
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)

    x = torch.stack(features)
    y = torch.tensor(labels, dtype=torch.long)

    print(f"x.shape: {x.shape}, edge_index.shape: {edge_index.shape}, y.shape: {y.shape}")
    print(f"Number of nodes: {G.number_of_nodes()}, Number of edges: {G.number_of_edges()}")

    return Data(x=x, edge_index=edge_index, y=y)

# 获取节点特征
train_data = prepare_data(train_graph, glove_embeddings)
test_data = prepare_data(test_graph, glove_embeddings)

# 设置掩码
train_data.train_mask = torch.tensor([True] * train_data.num_nodes, dtype=torch.bool)
test_data.test_mask = torch.tensor([True] * test_data.num_nodes, dtype=torch.bool)

# 创建数据加载器
train_loader = DataLoader([train_data], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data], batch_size=1, shuffle=False)

# 确保数据在同一设备上
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data = train_data.to(device)
test_data = test_data.to(device)


流式输出内容被截断，只能显示最后 5000 行内容。
Node 524976980799000578: label=True, feature_shape=torch.Size([300])
Node jazz has no label and will be skipped.
Node 500311883669585920: label=False, feature_shape=torch.Size([300])
Node georgia has no label and will be skipped.
Node Barnes has no label and will be skipped.
Node 524964457936863232: label=False, feature_shape=torch.Size([300])
Node 576889130258419712: label=False, feature_shape=torch.Size([300])
Node 536826643122503680: label=True, feature_shape=torch.Size([300])
Node cornelius gurlitt has no label and will be skipped.
Node Dresden has no label and will be skipped.
Node 524948790227517440: label=False, feature_shape=torch.Size([300])
Node 524993523746373633: label=False, feature_shape=torch.Size([300])
Node 553175714128990208: label=False, feature_shape=torch.Size([300])
Node 1 has no label and will be skipped.
Node gene has no label and will be skipped.
Node 544350565283532800: label=True, feature_shape=torch.Size([300])
Node 544357487911395

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [10]:

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, output_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, output_channels)  # 将输出维度调整为776

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.fc(x)  # 调整输出维度为776
        return F.log_softmax(x, dim=1)



In [ ]:
print("Unique labels in training data:", train_data.y.unique())
print("Unique labels in test data:", test_data.y.unique())


Unique labels in training data: tensor([0, 1], device='cuda:0')
Unique labels in test data: tensor([0, 1], device='cuda:0')


In [12]:
"""model = ComplexGCN(num_features=train_data.num_features, hidden_channels=128, output_channels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

def evaluate(loader):
    model.eval()
    correct = 0
    all_preds = []
    all_labels = []
    for data in loader:
        data = data.to(device)
        out = model(data)
        _, pred = out.max(dim=1)
        correct += int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        all_preds.extend(pred[data.test_mask].cpu().numpy())
        all_labels.extend(data.y[data.test_mask].cpu().numpy())
    accuracy = correct / len(all_labels)
    return accuracy, all_preds, all_labels

# 模型训练和数据分布打印
for epoch in range(200):
    model.train()
    for data in train_loader:
        data = data.to(device)  # 确保数据在同一设备上
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    # 评估训练集和测试集的准确率
    train_acc, _, _ = evaluate(train_loader)
    test_acc, _, _ = evaluate(test_loader)
    print(f'Epoch {epoch+1}, Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

    # 打印训练数据的分布情况
    print(f'Training Data Distribution at Epoch {epoch+1}')
    num_classes = data.y.max().item() + 1
    class_counts = [(data.y == i).sum().item() for i in range(num_classes)]
    print(f'Class Counts: {class_counts}')
    print(f'Node features: {data.x}')
    print(f'Edge index: {data.edge_index}')

# 最终评估并打印分类报告
_, all_preds, all_labels = evaluate(test_loader)
print('Final Test Accuracy: {:.4f}'.format(test_acc))
print(classification_report(all_labels, all_preds, target_names=['Non-Rumor', 'Rumor']))
"""
# 训练模型
from sklearn.metrics import classification_report
model = GCN(num_features=train_data.num_features, hidden_channels=128, output_channels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    for data in train_loader:
        data = data.to(device)  # 确保数据在同一设备上
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 评估模型并打印分类报告
model.eval()
all_preds = []
all_labels = []

for data in test_loader:
    data = data.to(device)  # 确保数据在同一设备上
    out = model(data)
    _, pred = out.max(dim=1)
    all_preds.extend(pred[data.test_mask].cpu().numpy())
    all_labels.extend(data.y[data.test_mask].cpu().numpy())

accuracy = (torch.tensor(all_preds) == torch.tensor(all_labels)).sum().item() / len(all_labels)
print('Accuracy: {:.4f}'.format(accuracy))

# 打印分类报告
print(classification_report(all_labels, all_preds, target_names=['Non-Rumor', 'Rumor']))

Epoch 1, Loss: 0.8244379758834839
Epoch 2, Loss: 2.23976731300354
Epoch 3, Loss: 0.7049151659011841
Epoch 4, Loss: 0.9507167935371399
Epoch 5, Loss: 0.9090463519096375
Epoch 6, Loss: 0.5892221331596375
Epoch 7, Loss: 0.6135815382003784
Epoch 8, Loss: 0.7267763018608093
Epoch 9, Loss: 0.5923232436180115
Epoch 10, Loss: 0.4499540328979492
Epoch 11, Loss: 0.38432130217552185
Epoch 12, Loss: 0.40769168734550476
Epoch 13, Loss: 0.4426378309726715
Epoch 14, Loss: 0.35670945048332214
Epoch 15, Loss: 0.30652910470962524
Epoch 16, Loss: 0.2508132755756378
Epoch 17, Loss: 0.20568709075450897
Epoch 18, Loss: 0.19547007977962494
Epoch 19, Loss: 0.18055349588394165
Epoch 20, Loss: 0.15085092186927795
Epoch 21, Loss: 0.14244282245635986
Epoch 22, Loss: 0.10591861605644226
Epoch 23, Loss: 0.07931370288133621
Epoch 24, Loss: 0.07251590490341187
Epoch 25, Loss: 0.06762382388114929
Epoch 26, Loss: 0.0711267963051796
Epoch 27, Loss: 0.04843157157301903
Epoch 28, Loss: 0.05778568238019943
Epoch 29, Loss: 

In [ ]:
print(edge_index)

NameError: name 'edge_index' is not defined